In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,VotingRegressor,BaggingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from catboost import CatBoost,CatBoostRegressor
from xgboost import XGBRegressor,XGBRFRegressor
from flaml import AutoML
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Apple-Stock-Price-Prediction-Sklearn'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
def valid(model,X,y,valid=False):
    preds = model.predict(X)
    if valid is True:
        results = {
            'MAE':mean_absolute_error(y_pred=preds,y_true=y),
            'MSE':mean_squared_error(y_pred=preds,y_true=y)
        }
    else:
        results = {
            'Val MAE':mean_absolute_error(y_pred=preds,y_true=y),
            'Val MSE':mean_squared_error(y_pred=preds,y_true=y)
        }
    return results
def train(model,X_train,X_test,y_train,y_test,name):
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    wandb.init(project=PROJECT_NAME,name=name)
    model.fit(X_train.reshape(-1,1),y_train.reshape(-1,1))
    wandb.log(valid(model,X_train.reshape(-1,1),y_train.reshape(-1,1),True))
    wandb.log(valid(model,X_test.reshape(-1,1),y_test.reshape(-1,1),False))
    wandb.sklearn.plot_regressor(model, X_train, X_test, y_train, y_test,  model_name=name)
    pickle.dump(model,open(f'./models/{name}.pkl','wb'))
    wandb.finish()
    return model

In [3]:
data = pd.read_csv('./data/data.csv')

In [4]:
X = data['Date']

In [5]:
y = data['Open']

In [6]:
preprocessings = (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)

In [7]:

decompositions = (
    PCA,
    KernelPCA
)

In [8]:
feature_selections = (
    VarianceThreshold,
    SelectKBest,
)

In [9]:
models = [
    ['KNeighborsRegressor',KNeighborsRegressor],
    ['LogisticRegression',LogisticRegression],
    ['LogisticRegressionCV',LogisticRegressionCV],
    ['DecisionTreeRegressor',DecisionTreeRegressor],
    ['GradientBoostingRegressor',GradientBoostingRegressor],
    ['AdaBoostRegressor',AdaBoostRegressor],
    ['RandomForestRegressor',RandomForestRegressor],
    ['BaggingRegressor',BaggingRegressor],
    ['GaussianNB',GaussianNB],
    ['ExtraTreesRegressor',ExtraTreesRegressor],
    ['CatBoost',CatBoost],
    ['CatBoostRegressor',CatBoostRegressor],
    ['XGBRegressor',XGBRegressor],
    ['XGBRFRegressor',XGBRFRegressor],
    ['ExtraTreesRegressor',ExtraTreesRegressor],
]

In [10]:
new_X = []
for X_batch in X:
    X_batch = X_batch.split('-')
    X_batch = f'{X_batch[0]}{X_batch[1]}{X_batch[2]}'
    X_batch = int(X_batch)
    new_X.append(X_batch)

In [11]:
X = pd.DataFrame(new_X,columns=['Date'])['Date']

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125)

In [13]:
X_train_old = X_train.copy()
X_test_old = X_test.copy()

In [14]:
train(GradientBoostingRegressor(),X_train,X_test,y_train,y_test,name='baseline')

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


AttributeError: 'numpy.ndarray' object has no attribute 'predict'

In [ ]:
for preprocessing in preprocessings:
    X_train = X_train_old.copy()
    X_test = X_test_old.copy()
    preprocessing = preprocessing()
    X_train = preprocessing.fit_transform(X_train)
    X_test = preprocessing.transform(X_test)
    train(GradientBoostingRegressor(),X_train,X_test,y_train,y_test,name=f'{preprocessing}-preprocessing')

In [ ]:
for decomposition in decompositions:
    X_train = X_train_old.copy()
    X_test = X_test_old.copy()
    decomposition = decomposition()
    X_train = decomposition.fit_transform(X_train)
    X_test = decomposition.transform(X_test)
    train(GradientBoostingRegressor(),X_train,X_test,y_train,y_test,name=f'{decomposition}-decomposition')

In [ ]:
for feature_selection in feature_selections:
    X_train = X_train_old.copy()
    X_test = X_test_old.copy()
    feature_selection = feature_selection()
    X_train = feature_selection.fit_transform(X_train)
    X_test = feature_selection.transform(X_test)
    train(GradientBoostingRegressor(),X_train,X_test,y_train,y_test,name=f'{feature_selection}-feature_selection')

In [ ]:
# for model in models:
#     train(model(),X_train,X_test,y_train,y_test,name=f'{model}-model')